Installing and Importing Libraries

In [2]:
pip install beautifulsoup4

     |████████████████████████████████| 122kB 9.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install geocoder

     |████████████████████████████████| 102kB 5.7MB/s ta 0:00:011
     |████████████████████████████████| 829kB 6.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [11]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import geocoder # import geocoder

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [5]:
tables = soup.find_all('table')

In [6]:
#Create array to hold the data we extract
postal = []
borough = []
neighbourhood = []

# Only 1st table contains data, we desire for this project
rows = tables[0].find_all('tr')

for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        postal.append(cells[0].text.strip())
        borough.append(cells[1].text.strip())
        neighbourhood.append(cells[2].text.strip())

In [19]:
# Creating a desired dataframe
data = np.array([postal, borough, neighbourhood])
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_values = ['Postal Code', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data = data.T, columns = column_values)

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

# Display after resetting Index
df = df.reset_index(drop = True)
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
no_of_rows = df.shape[0]
no_of_rows

103

In [21]:
# creating a dataframe that has the geographical coordinates of each postal code
cord_df = pd.read_csv('https://cocl.us/Geospatial_data')

# Merging the two dataframes to get desired output
merge_df = pd.merge(df, cord_df, how='outer', on=["Postal Code"])
merge_df.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
